<a href="https://colab.research.google.com/github/tusharkanta/ML_DL/blob/eva/S4/S4_Soln.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, 3, padding=1) #input -? OUtput? 28
        self.conv1_bn = nn.BatchNorm2d(8) 
        self.conv2 = nn.Conv2d(8, 8, 3, padding=1) #28
        self.conv2_bn = nn.BatchNorm2d(8)
        self.pool1 = nn.MaxPool2d(2, 2) #14
        self.dropout1 = nn.Dropout(0.25)
        self.conv3 = nn.Conv2d(8, 16, 3,padding=1) #14
        self.conv3_bn = nn.BatchNorm2d(16)
        
        
        self.conv4 = nn.Conv2d(16, 16, 3,padding=1) #14
        self.conv4_bn = nn.BatchNorm2d(16)
        self.pool2 = nn.MaxPool2d(2, 2) #7
        self.dropout2 = nn.Dropout(0.25)
        self.conv5 = nn.Conv2d(16, 16, 3) #5
        self.conv5_bn = nn.BatchNorm2d(16)
        self.conv6 = nn.Conv2d(16, 32, 3) #3
        self.conv7 = nn.Conv2d(32, 10, 3)

    def forward(self, x):
        x = self.dropout1(self.pool1(self.conv2_bn(F.relu(self.conv2(self.conv1_bn(F.relu(self.conv1(x))))))))
        x = self.dropout2(self.pool2(self.conv4_bn(F.relu(self.conv4(self.conv3_bn(F.relu(self.conv3(x))))))))
        x = self.conv5_bn(F.relu(self.conv5(x)))
        x = F.relu(self.conv6(x))
        x = self.conv7(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [168]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              80
       BatchNorm2d-2            [-1, 8, 28, 28]              16
            Conv2d-3            [-1, 8, 28, 28]             584
       BatchNorm2d-4            [-1, 8, 28, 28]              16
         MaxPool2d-5            [-1, 8, 14, 14]               0
           Dropout-6            [-1, 8, 14, 14]               0
            Conv2d-7           [-1, 16, 14, 14]           1,168
       BatchNorm2d-8           [-1, 16, 14, 14]              32
            Conv2d-9           [-1, 16, 14, 14]           2,320
      BatchNorm2d-10           [-1, 16, 14, 14]              32
        MaxPool2d-11             [-1, 16, 7, 7]               0
          Dropout-12             [-1, 16, 7, 7]               0
           Conv2d-13             [-1, 16, 5, 5]           2,320
      BatchNorm2d-14             [-1, 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:


torch.manual_seed(1)
batch_size = 512

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [171]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(16):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)















  0%|          | 0/118 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.














loss=2.3365349769592285 batch_id=0:   0%|          | 0/118 [00:00<?, ?it/s]













loss=2.3365349769592285 batch_id=0:   1%|          | 1/118 [00:00<00:17,  6.51it/s]













loss=2.3274006843566895 batch_id=1:   1%|          | 1/118 [00:00<00:17,  6.51it/s]













loss=2.320668935775757 batch_id=2:   1%|          | 1/118 [00:00<00:17,  6.51it/s] 













loss=2.320668935775757 batch_id=2:   3%|▎         | 3/118 [00:00<00:14,  7.75it/s]













loss=2.305062770843506 batch_id=3:   3%|▎         | 3/118 [00:00<00:14,  7.75it/s]













loss=2.2816174030303955 batch_id=4:   3%|▎         | 3/118 [00:00<00:14,  7.75it/s]













loss=2.2816174030303955 batch_id=4:   4%|▍         | 5/118 [00:00<00:12,  


Test set: Average loss: 0.1178, Accuracy: 9664/10000 (96.64%)

















loss=0.12307340651750565 batch_id=0:   0%|          | 0/118 [00:00<?, ?it/s]













loss=0.12307340651750565 batch_id=0:   1%|          | 1/118 [00:00<00:18,  6.26it/s]













loss=0.1202930435538292 batch_id=1:   1%|          | 1/118 [00:00<00:18,  6.26it/s] 













loss=0.12286949157714844 batch_id=2:   1%|          | 1/118 [00:00<00:18,  6.26it/s]













loss=0.12286949157714844 batch_id=2:   3%|▎         | 3/118 [00:00<00:15,  7.46it/s]













loss=0.1558787077665329 batch_id=3:   3%|▎         | 3/118 [00:00<00:15,  7.46it/s] 













loss=0.12896685302257538 batch_id=4:   3%|▎         | 3/118 [00:00<00:15,  7.46it/s]













loss=0.12896685302257538 batch_id=4:   4%|▍         | 5/118 [00:00<00:13,  8.46it/s]













loss=0.16267544031143188 batch_id=5:   4%|▍         | 5/118 [00:00<00:13,  8.46it/s]













loss=0.14443007111549377 batch_id=6:   4%|▍         | 5/118 [00:00<00:13,  8.46it/s]













loss=0.1444300


Test set: Average loss: 0.0617, Accuracy: 9812/10000 (98.12%)

















loss=0.11781665682792664 batch_id=0:   0%|          | 0/118 [00:00<?, ?it/s]













loss=0.11781665682792664 batch_id=0:   1%|          | 1/118 [00:00<00:16,  7.03it/s]













loss=0.09769413620233536 batch_id=1:   1%|          | 1/118 [00:00<00:16,  7.03it/s]













loss=0.11745984107255936 batch_id=2:   1%|          | 1/118 [00:00<00:16,  7.03it/s]













loss=0.11745984107255936 batch_id=2:   3%|▎         | 3/118 [00:00<00:14,  8.14it/s]













loss=0.10968133807182312 batch_id=3:   3%|▎         | 3/118 [00:00<00:14,  8.14it/s]













loss=0.12963780760765076 batch_id=4:   3%|▎         | 3/118 [00:00<00:14,  8.14it/s]













loss=0.12963780760765076 batch_id=4:   4%|▍         | 5/118 [00:00<00:12,  9.34it/s]













loss=0.09526748955249786 batch_id=5:   4%|▍         | 5/118 [00:00<00:12,  9.34it/s]













loss=0.09865139424800873 batch_id=6:   4%|▍         | 5/118 [00:00<00:12,  9.34it/s]













loss=0.0986513

...


Test set: Average loss: 0.0247, Accuracy: 9929/10000 (99.29%)

















loss=0.04666002839803696 batch_id=0:   0%|          | 0/118 [00:00<?, ?it/s]













loss=0.04666002839803696 batch_id=0:   1%|          | 1/118 [00:00<00:16,  6.91it/s]













loss=0.05302499607205391 batch_id=1:   1%|          | 1/118 [00:00<00:16,  6.91it/s]













loss=0.0390901193022728 batch_id=2:   1%|          | 1/118 [00:00<00:16,  6.91it/s] 













loss=0.0390901193022728 batch_id=2:   3%|▎         | 3/118 [00:00<00:14,  8.16it/s]













loss=0.04091436415910721 batch_id=3:   3%|▎         | 3/118 [00:00<00:14,  8.16it/s]













loss=0.037882670760154724 batch_id=4:   3%|▎         | 3/118 [00:00<00:14,  8.16it/s]













loss=0.037882670760154724 batch_id=4:   4%|▍         | 5/118 [00:00<00:12,  9.36it/s]













loss=0.06806330382823944 batch_id=5:   4%|▍         | 5/118 [00:00<00:12,  9.36it/s] 













loss=0.024373553693294525 batch_id=6:   4%|▍         | 5/118 [00:00<00:12,  9.36it/s]













loss=0.0243


Test set: Average loss: 0.0239, Accuracy: 9926/10000 (99.26%)

















loss=0.04419245570898056 batch_id=0:   0%|          | 0/118 [00:00<?, ?it/s]













loss=0.04419245570898056 batch_id=0:   1%|          | 1/118 [00:00<00:16,  7.12it/s]













loss=0.03715064749121666 batch_id=1:   1%|          | 1/118 [00:00<00:16,  7.12it/s]













loss=0.05474916100502014 batch_id=2:   1%|          | 1/118 [00:00<00:16,  7.12it/s]













loss=0.05474916100502014 batch_id=2:   3%|▎         | 3/118 [00:00<00:13,  8.22it/s]













loss=0.07029599696397781 batch_id=3:   3%|▎         | 3/118 [00:00<00:13,  8.22it/s]













loss=0.03711245208978653 batch_id=4:   3%|▎         | 3/118 [00:00<00:13,  8.22it/s]













loss=0.03711245208978653 batch_id=4:   4%|▍         | 5/118 [00:00<00:12,  9.38it/s]













loss=0.053672052919864655 batch_id=5:   4%|▍         | 5/118 [00:00<00:12,  9.38it/s]













loss=0.029487244784832 batch_id=6:   4%|▍         | 5/118 [00:00<00:12,  9.38it/s]   













loss=0.02948


Test set: Average loss: 0.0228, Accuracy: 9928/10000 (99.28%)

